1. Choose a Python library

Two common ways to get FRED data programmatically:
	•	fredapi — dedicated wrapper for FRED’s API. Requires a free API key from St. Louis Fed.
	•	pandas_datareader — can also pull FRED series directly without an API key (though sometimes rate-limited).

For a pipeline that you’ll rerun regularly, fredapi is more robust.

2. Get a FRED API key

Go to https://fred.stlouisfed.org/docs/api/api_key.html, sign up (free), and copy your key.
Store it as an environment variable in your .env file or in your shell: 

export FRED_API_KEY="your_api_key_here"

This keeps it out of your code repository.

3. Install the library
   pip install fredapi pandas
4. Basic code to fetch series
   from fredapi import Fred
    import pandas as pd
    import os

    fred = Fred(api_key=os.getenv("FRED_API_KEY"))

    # choose some series IDs
    series_ids = {
        "10y_treasury": "DGS10",
        "2y_treasury": "DGS2",
        "high_yield_spread": "BAMLH0A0HYM2",
        "unemployment_rate": "UNRATE"
    }

    # download each as a dataframe and join
    dfs = []
    for name, sid in series_ids.items():
        data = fred.get_series(sid)  # returns pandas Series with Date index
        df = data.to_frame(name)
        dfs.append(df)

    market_data = pd.concat(dfs, axis=1)

    # optional: resample monthly to match loan origination
    market_monthly = market_data.resample('M').last()

    market_monthly.to_csv("market_variables.csv")

    This gives you a single CSV of macro/market variables indexed by date. Your loan dataset can then be merged on origination_date rounded to month.

5. Make it a “pipeline”

Put the above code into a script (e.g. fetch_market_data.py).
Every time you run it, it refreshes the CSV.

If you want it automated:
	•	Use a cron job or GitHub Actions to run the script weekly.
	•	Or add it as a notebook cell in your main data prep notebook so it always fetches the latest.

⸻

6. Extra touches
	•	Parameterize: Put the series IDs in a YAML/JSON file so you can add/remove variables easily.
	•	Logging: Use logging to print when each series was last updated.
	•	Data integrity: Save raw pulls in a /raw_data folder with a timestamp before cleaning.